In [21]:
from functions import Model, get99, reg_train_test, splitData, dropMissingValues, z_score, outliers_knn, outliers_dbscan, getNoise, getRelFeatures, drop_features, getCombinations, get_unique_list, bold, blue, red, green, getBestModel
import pandas as pd
from sklearn.linear_model import LinearRegression
import warnings
import json
import numpy as np
warnings.filterwarnings('ignore')

In [22]:
# JSON file
f = open ('data/json/input.json', "r")
  # Reading from file
PARAMETERS = json.loads(f.read())

SPLIT_SIZE = PARAMETERS["SPLIT_SIZE"]

OUTLIER99 = PARAMETERS["OUTLIER99"] 
OUTLIER_ZSCORE = {"STD" : PARAMETERS["OUTLIER_ZSCORE"]["STD"]}
OUTLIER_KNN = {"K": PARAMETERS["OUTLIER_KNN"]["K"], "X": PARAMETERS["OUTLIER_KNN"]["X"]}
OUTLIER_DBSCAN = {"K": PARAMETERS["OUTLIER_DBSCAN"]["K"], "X": PARAMETERS["OUTLIER_DBSCAN"]["X"], "EPS": PARAMETERS["OUTLIER_DBSCAN"]["EPS"], "MIN_SAMPLES":  PARAMETERS["OUTLIER_DBSCAN"]["MIN_SAMPLES"]}
OUTLIER_NOISE = {"CV": PARAMETERS["OUTLIER_NOISE"]["CV"]}

f.close()

#TODO FEATURE
#TODO Regressionmodels
#TODO TESTDATA --> path to csv-file


In [23]:
#Get raw Data
df = pd.read_csv("data/csv/house_data_training.csv", sep=';') 
# remove unnamed column
df = df.iloc[:, 1:]
#Transform string to datetime
df["date"] = pd.to_datetime(df["date"])

maeList = []#for model selection

In [24]:
df = dropMissingValues(df) 

In [25]:
# Outlier Detection
outlier_lists_all = []
outlier_dict_all = {}

if OUTLIER99 == True:
    print("outlier_99")
    outlier_list_99 = get99(df) #TODO Mean for price for trainingsdata
    #outlier_lists_all.append(outlier_list_99)
    outlier_dict_all["99_mean"] = outlier_list_99
    #outlier_lists_all.append(outlier_list_99)
    outlier_dict_all["99_drop"] = outlier_list_99
    print("done\n")

print('starting z_score')
for p in OUTLIER_ZSCORE["STD"]:
    print(p)
    outlier_list_z_score = z_score(df, p)
    #outlier_lists_all.append(outlier_list_z_score)
    outlier_dict_all["z_score_"+str(p)] = outlier_list_z_score

print('done \n\nstarting knn')
for k in OUTLIER_KNN["K"]:
    for o in OUTLIER_KNN["X"]:
        print(k, o)
        outlier_list_knn = outliers_knn(df, k, o)
        #outlier_lists_all.append(outlier_list_knn)
        outlier_dict_all["knn_"+str(k)+"_"+str(o)] = outlier_list_knn

print('done \n\nstarting db_scan')
outlier_list_dbscan = outliers_dbscan(df) #TODO multiple parameters
#outlier_lists_all.append(outlier_list_dbscan)
outlier_dict_all["db_scan"] = outlier_list_dbscan

print('done \n\nstarting noise')
for cv in OUTLIER_NOISE["CV"]:
        print(cv)
        outlier_list_noise = getNoise(df, cv)
        #outlier_lists_all.append(outlier_list_noise)
        outlier_dict_all["noise_"+str(cv)] = outlier_list_noise
print('done')

# outlier_lists_all = [outlier_list_99, outlier_list_z_score, outlier_list_knn, outlier_list_dbscan, outlier_list_noise]

outlier_99
starting z_score
3
10
done 

 starting knn
3 200
3 1000
5 200
5 1000
done 

 starting db_scan
done 

 starting noise
3
5
done


In [43]:
outlier_lists_all = list(outlier_dict_all.values())
outlier_lists_keys = list(outlier_dict_all.keys())
combination_list = []
for i, combo in enumerate(getCombinations(outlier_lists_keys), 1):
    
    if not i == 1: #emtpy tuple
        combination_list.append(combo)
        # print('combo #{}: {}'.format(i, combo))
print("...")
for c in combination_list[10:]:
    print(bold("Combination: "), blue(c),"\n",)
print(bold(len(combination_list)), " total Combinations")

...
Combination:  ('noise_5',) 

Combination:  ('99_mean', '99_drop') 

Combination:  ('99_mean', 'z_score_3') 

Combination:  ('99_mean', 'z_score_10') 

Combination:  ('99_mean', 'knn_3_200') 

Combination:  ('99_mean', 'knn_3_1000') 

Combination:  ('99_mean', 'knn_5_200') 

Combination:  ('99_mean', 'knn_5_1000') 

Combination:  ('99_mean', 'db_scan') 

Combination:  ('99_mean', 'noise_3') 

Combination:  ('99_mean', 'noise_5') 

Combination:  ('99_drop', 'z_score_3') 

Combination:  ('99_drop', 'z_score_10') 

Combination:  ('99_drop', 'knn_3_200') 

Combination:  ('99_drop', 'knn_3_1000') 

Combination:  ('99_drop', 'knn_5_200') 

Combination:  ('99_drop', 'knn_5_1000') 

Combination:  ('99_drop', 'db_scan') 

Combination:  ('99_drop', 'noise_3') 

Combination:  ('99_drop', 'noise_5') 

Combination:  ('z_score_3', 'z_score_10') 

Combination:  ('z_score_3', 'knn_3_200') 

Combination:  ('z_score_3', 'knn_3_1000') 

Combination:  ('z_score_3', 'knn_5_200') 

Combination:  ('z_scor

In [27]:
model_obj_list = []
for i, c in enumerate(combination_list):
    print(f'\n{bold(i)}: {blue(c)}')
    outlier_list = get_unique_list(outlier_dict_all, c)
    try: 
        if "99_mean" in c:
            X_train, X_test, y_train, y_test = splitData(df, SPLIT_SIZE, outlier_list, "both", outlier_list_99)
        else:
            X_train, X_test, y_train, y_test = splitData(df, SPLIT_SIZE, outlier_list, "drop")

        #TODO Regressionmodels
        model = LinearRegression()
        obj = Model(model, (X_train, X_test, y_train, y_test), c)
        model_obj_list.append(obj)

    except Exception as e:
        print(bold("Error"), red(e))


0: ('99_mean',)
dropped 0 / 432 rows
replaced 316 / 432 rows

1: ('99_drop',)
dropped 316 / 316 rows
replaced 0 / 316 rows

2: ('z_score_3',)
dropped 1288 / 1717 rows
replaced 0 / 1717 rows

3: ('z_score_10',)
dropped 39 / 50 rows
replaced 0 / 50 rows

4: ('knn_3_200',)
Error 'numpy.ndarray' object has no attribute 'remove'

5: ('knn_3_1000',)
Error 'numpy.ndarray' object has no attribute 'remove'

6: ('knn_5_200',)
Error 'numpy.ndarray' object has no attribute 'remove'

7: ('knn_5_1000',)
Error 'numpy.ndarray' object has no attribute 'remove'

8: ('db_scan',)
dropped 2582 / 2762 rows
replaced 0 / 2762 rows

9: ('noise_3',)
dropped 312 / 426 rows
replaced 0 / 426 rows

10: ('noise_5',)
dropped 313 / 427 rows
replaced 0 / 427 rows

11: ('99_mean', '99_drop')
dropped 0 / 316 rows
replaced 316 / 316 rows

12: ('99_mean', 'z_score_3')
dropped 980 / 1296 rows
replaced 316 / 1296 rows

13: ('99_mean', 'z_score_10')
dropped 38 / 354 rows
replaced 316 / 354 rows

14: ('99_mean', 'knn_3_200')


In [28]:
#Feature Selection
list_relevant_features = getRelFeatures(df)
#df_dropped_features = drop_features(df, list_relevant_features)

In [29]:
#Summary
df_summary = pd.DataFrame(columns=['combo', 'mae', 'score'])
for o in model_obj_list:
    #print(f'{o.get_type()}, {o.get_comb()}')
    o.train()
    mae, score = o.summary()
    comb = o.get_comb()
    row = {'combo': comb, 'mae': mae, 'score': score}
    df_summary = df_summary.append(row, ignore_index=True)

df_summary.sort_values("mae")

,combo,mae,score
5,"(noise_3,)",6283429.62,0.049511
1,"(99_drop,)",6283726.02,0.049517
6,"(noise_5,)",6284195.62,0.049519
61,"(noise_3, noise_5)",6285263.16,0.049514
24,"(99_drop, noise_3)",6285818.65,0.049508
...,...,...,...
1855,"(99_mean, 99_drop, z_score_3, knn_3_200, knn_3...",6508708.58,0.04916
1859,"(99_mean, 99_drop, z_score_3, knn_3_200, knn_5...",6508708.58,0.04916
1860,"(99_mean, 99_drop, z_score_3, knn_3_200, knn_5...",6508708.58,0.04916
1989,"(99_mean, 99_drop, z_score_3, z_score_10, knn_...",6508708.58,0.04916


In [30]:
#row
mae_best = df_summary[df_summary.mae==df_summary.mae.min()]
score_best = df_summary[df_summary.score==df_summary.score.max()]
print(f'{bold("Lowest mae:")} {mae_best}')
print(f'\n{bold("Highest score:")} {score_best}')

#df_summary["mae"].min()

Lowest mae:         combo         mae     score
5  (noise_3,)  6283429.62  0.049511

Highest score:                                               combo         mae     score
41                          (knn_3_200, knn_3_1000)  6355882.29  0.049631
43                          (knn_3_200, knn_5_1000)  6355882.29  0.049631
47                          (knn_3_1000, knn_5_200)  6355882.29  0.049631
48                         (knn_3_1000, knn_5_1000)  6355882.29  0.049631
52                          (knn_5_200, knn_5_1000)  6355882.29  0.049631
192              (knn_3_200, knn_3_1000, knn_5_200)  6355882.29  0.049631
193             (knn_3_200, knn_3_1000, knn_5_1000)  6355882.29  0.049631
197              (knn_3_200, knn_5_200, knn_5_1000)  6355882.29  0.049631
207             (knn_3_1000, knn_5_200, knn_5_1000)  6355882.29  0.049631
522  (knn_3_200, knn_3_1000, knn_5_200, knn_5_1000)  6355882.29  0.049631


In [31]:
best_model_obj = getBestModel(model_obj_list, df_summary, mae_best.index[0])
print(best_model_obj)

Model: LinearRegression() Combination: ('noise_3',)
